<a href="https://colab.research.google.com/github/RandomForestRanger/PhD/blob/master/006_Messaround_with_Bernoulli_NB_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pip installation
!pip install --upgrade -q gspread
!pip install gspread-dataframe
!pip install -U scikit-learn>=0.20
!pip install --upgrade scikit-learn

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from sklearn.naive_bayes import ComplementNB

import sklearn.model_selection as ms
import sklearn.metrics as sklm

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MultiLabelBinarizer # This one to later turn my LabelsNP into a onehot encoded 2d array

from sklearn import preprocessing
from sklearn.naive_bayes import BernoulliNB  # this will be operative today...
from sklearn.linear_model import LogisticRegression #Mb Added, not sure it should be here... 
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import numpy.random as nr

%matplotlib inline

In [ ]:
#authorise node to access Gdrive via KDL
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
# now containing "6 short munged and num only 30 Oct IRIS dataset"

#link to pre-opened csv file in my gdrive 
data1 = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Q2YwmFiFSOkW_V9cRYgZ4GdGGl63_N6mOejEJQB1e0M/edit#gid=835063997')

In [ ]:
#get the data as a worksheet  
ws = data1.worksheet('icanhasdata')

In [ ]:
#convert the data to pandas dataframe (can include argument , header = None   if this is needed)
FullDF = get_as_dataframe(ws)
#inspect data
FullDF.tail()

In [ ]:
#since g-sheets brings along a lot of empty/Nan cellumns or rows, this chops them out. Can replace "all" with "any" if I want to discard any incomplete collumns or rows

# I changed the "all" to "any", to root out all cases of NaN...

FullDF.dropna(axis=1, how='all' ,inplace=True )
FullDF.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
FullDF.tail()

# if there are still "NaN" values, I want to fill them with "0". Go about this thusly:

FullDF.fillna(0)

In [ ]:
print(FullDF.shape)

In [ ]:
FeatDF  = FullDF.drop( columns = "PDV")
LabelDF = FullDF["PDV"]
print(FeatDF.shape)
print(LabelDF.shape)

In [ ]:
# Convent the DF to a NP array

FeatNP  = FeatDF.values
LabelNP = LabelDF.values
print("Feat:")
print(FeatNP.shape)
print(FeatNP)
print("Label:")
print(LabelNP.shape)

In [ ]:
print(FeatNP[0:1,0:])

In [ ]:
## Randomly sample cases to create independent training and test data
nr.seed(1)
indx = range(FeatNP.shape[0])
indx = ms.train_test_split(indx, test_size = 100)
X_train = FeatNP[indx[0],:]
y_train = np.ravel(LabelNP[indx[0]])
X_test = FeatNP[indx[1],:]
y_test = np.ravel(LabelNP[indx[1]])

In [ ]:
print("X train shape is: ", X_train.shape, "\n")
print("X test shape is: ", y_test.shape, "\n")
print("Y train shape is: ", y_train.shape, "\n")
print("Y test shape is: ", y_test.shape, "\n")
print("Unscaled X train record number 700 looks like this: \n", X_train[700:701,:])
print(np.isnan(X_train.any()))
print(np.isfinite(X_train.any()))

In [ ]:
## ONLY scale if I'm going to Gaussian...
#scale = preprocessing.StandardScaler()
#scale.fit(X_train)
#X_train = scale.transform(X_train)

In [ ]:
#print("Scaled X train record number 700 looks like this: \n", X_train[700:701,:])

In [ ]:
NB_mod = BernoulliNB()

##from sklearn.naive_bayes import MultinomialNB
##NB_mod = MultinomialNB()
##MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
###NB_mod = ComplementNB()

NB_mod.fit(X_train, y_train)
BernoulliNB(alpha=1.0, class_prior=None, fit_prior=True, )

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
#   X_test = scale.transform(X_test)
scores = NB_mod.predict(X_test)

In [ ]:
## What, this worked ?!!   ##
# MB trying to figure out how to see the coefficients/parameters...
print(NB_mod.coef_)
 

In [ ]:
y_test

array(['V', 'NP', 'V', 'P', 'NP', 'P', 'D', 'D', 'D', 'V', 'P', 'NP', 'P',
       'P', 'P', 'P', 'NP', 'D', 'P', 'NP', 'D', 'P', 'NP', 'NP', 'V',
       'P', 'NP', 'D', 'P', 'NP', 'P', 'NP', 'NP', 'NP', 'NP', 'P', 'P',
       'P', 'NP', 'NP', 'NP', 'NP', 'V', 'NP', 'NP', 'P', 'P', 'NP', 'NP',
       'D', 'P', 'NP', 'NP', 'P', 'NP', 'D', 'NP', 'D', 'NP', 'NP', 'P',
       'NP', 'NP', 'P', 'V', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'D', 'P',
       'P', 'NP', 'V', 'NP', 'NP', 'D', 'NP', 'D', 'P', 'V', 'NP', 'NP',
       'P', 'P', 'P', 'P', 'P', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'NP',
       'NP', 'NP', 'D'], dtype=object)

In [ ]:
scores

array(['D', 'NP', 'P', 'P', 'NP', 'NP', 'D', 'NP', 'P', 'V', 'NP', 'NP',
       'NP', 'P', 'P', 'NP', 'NP', 'NP', 'P', 'P', 'V', 'P', 'NP', 'NP',
       'P', 'P', 'P', 'NP', 'P', 'NP', 'P', 'P', 'NP', 'NP', 'NP', 'P',
       'NP', 'P', 'NP', 'NP', 'NP', 'NP', 'V', 'NP', 'NP', 'P', 'NP',
       'NP', 'NP', 'D', 'P', 'NP', 'NP', 'P', 'NP', 'P', 'NP', 'P', 'NP',
       'NP', 'D', 'V', 'NP', 'P', 'D', 'NP', 'D', 'P', 'P', 'NP', 'V',
       'NP', 'P', 'NP', 'P', 'V', 'NP', 'NP', 'V', 'NP', 'D', 'P', 'V',
       'NP', 'D', 'P', 'NP', 'NP', 'P', 'P', 'NP', 'P', 'NP', 'NP', 'V',
       'P', 'P', 'NP', 'NP', 'P'], dtype='<U2')

In [ ]:
def print_metrics_4(labels, scores):
   
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score NP         Score P          Score D          Score V')
    print('Actual NP      %6d' % conf[0,0] + '            %5d' % conf[0,1] + '             %5d' % conf[0,2]+ '             %5d' % conf[0,3])
    print('Actual P       %6d' % conf[1,0] + '            %5d' % conf[1,1] + '             %5d' % conf[1,2]+ '             %5d' % conf[1,3])
    print('Actual D       %6d' % conf[2,0] + '            %5d' % conf[2,1] + '             %5d' % conf[2,2]+ '             %5d' % conf[2,3])
    print('Actual V       %6d' % conf[3,0] + '            %5d' % conf[3,1] + '             %5d' % conf[3,2]+ '             %5d' % conf[3,3])
    ## Now compute and display the accuracy and metrics
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('            NP        P          D          V')
    print('Num case    %0.2f' % metrics[3][0] + '     %0.2f' % metrics[3][1] + '      %0.2f' % metrics[3][2]+ '      %0.2f' % metrics[3][3])
    print('Precision   %0.2f' % metrics[0][0] + '      %0.2f' % metrics[0][1] + '       %0.2f' % metrics[0][2]+ '       %0.2f' % metrics[0][3])
    print('Recall      %0.2f' % metrics[1][0] + '      %0.2f' % metrics[1][1] + '       %0.2f' % metrics[1][2]+ '       %0.2f' % metrics[1][3])
    print('F1          %0.2f' % metrics[2][0] + '      %0.2f' % metrics[2][1] + '       %0.2f' % metrics[2][2]+ '       %0.2f' % metrics[2][3])
    
print_metrics_4(y_test, scores) 

# with 110 variables, GaussianNB   accuracy is 0.12
# with 89 variables,  GaussianNB   accuracy is 0.20 
# with 89 variables,  ComplementNB accuracy is 0.33
# with 49 variables,  ComplementNB accuracy is 0.55
# when i cut out the Unique ID, accuracy moved up to 0.58
# Using Bernoulli, I move this up to 0.64 ...(should I cross-validate?)  ...should I run this more than once?

                 Confusion matrix
                 Score NP         Score P          Score D          Score V
Actual NP           3                4                 4                 2
Actual P            1               35                 6                 2
Actual D            2               10                22                 1
Actual V            2                0                 2                 4

Accuracy        0.64
 
            NP        P          D          V
Num case    13.00     44.00      35.00      8.00
Precision   0.38      0.71       0.65       0.44
Recall      0.23      0.80       0.63       0.50
F1          0.29      0.75       0.64       0.47
